In [1]:
%matplotlib inline
import pandas as pd
import os
import sys
sys.path.append('..')
from src.fixed_path_gas_station import fixed_path_gas_station as fpgs

In [2]:
ROUTE_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten','Fahrzeugrouten')
BERTA_ROUTE_PATH = os.path.join(ROUTE_PATH,  'Bertha Benz Memorial Route.csv')

GAS_STATIONS_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten', 'Tankstellen.csv')
GAS_PRICE_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten', 'Benzinpreise')

In [3]:
with open(BERTA_ROUTE_PATH,'r') as f:
    capacity = float(f.readline())
bertas_route = pd.read_csv(BERTA_ROUTE_PATH, names=['Timestamp_str', 'Gas_Station_Id'], sep=';',skiprows=1)
bertas_route['Timestamp'] = bertas_route['Timestamp_str'].apply(lambda x: pd.Timestamp(x))

In [4]:
gas_stations_df = pd.read_csv(GAS_STATIONS_PATH, sep=';', names=['id', 'Name', 'Company', 'Street', 'House_Number', 'Postalcode', 'City', 'Lat', 'Long'],index_col='id')

In [5]:
def load_price(gas_station_id, timestamp):
    try:
        gas_station = pd.read_csv(os.path.join(GAS_PRICE_PATH,'{}.csv'.format(gas_station_id)), names=['Timestamp', 'Price'],  index_col='Timestamp',parse_dates=['Timestamp'],sep=';')
    except FileNotFoundError:
        raise ValueError('You tried to access gas station with id {}, but it was not in the data.'.format(gas_station_id))
    return gas_station[gas_station.index <= timestamp].iloc[-1]['Price']

def millieuro2euro(millieuro):
    return millieuro * 0.001

In [6]:
cost = []
coordinates = []

for index, row in bertas_route.iterrows():
    cost.append(load_price(row['Gas_Station_Id'],row['Timestamp']))
    coordinates.append(fpgs.Coordinate(gas_stations_df.loc[row['Gas_Station_Id']]['Lat']
                                    ,gas_stations_df.loc[row['Gas_Station_Id']]['Long']))
bertas_route['cost'] = cost
bertas_route['coords'] = coordinates

In [7]:
start_fuel = 0
result = fpgs.FixedPathGasStation(bertas_route, capacity, start_fuel)
bertas_route['fill_liters'] = result.fill_liters
bertas_route['dist_to_next'] = result.segment_distance

In [8]:
"Berta will pay {}€".format(millieuro2euro(result.price)) # should be 14.340753951663311

'Berta will pay 14.340753951663311€'

In [9]:
output=bertas_route[bertas_route['fill_liters']!=0][['Timestamp_str','Gas_Station_Id','cost','fill_liters']]
output.to_csv(os.path.join(ROUTE_PATH,'result.csv'),index=False,sep=';',header=False)

## Map

In [10]:
color = {
    'light': '#8DC7B1',
    'red':'#DD000B',
    'green':'#C1FF00',
    'blue':'#00B7FF',
    'dark':'#2C3D49'
}

In [11]:
from ipyleaflet import Map, Polyline, CircleMarker, Circle

m = Map(center=list(bertas_route.iloc[0]['coords']), zoom=10)

path = [list(row['coords']) for i, row in bertas_route.iterrows()]
p = Polyline(locations=path)
p.fill_opacity = 0.0
p.color=color['dark']
m.add_layer(p)

for index, row in bertas_route.iterrows():
    cm = Circle(location=list(row['coords']), radius=500, weight=3,
                  color=color['light'], fill_color=color['light'], opacity=1.0, fill_opacity=0.5)
    if result.fill_command_km[index] == 'fill up':
        cm.color=color['green']
        cm.fill_color=color['green']
    elif result.fill_liters[index] != 0.0:
        cm.color=color['blue']
        cm.fill_color=color['blue']
    path.append(list(row['coords']))
    m.add_layer(cm)
m

Map(center=[49.54950147, 8.449381135], layers=(TileLayer(attribution='Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors', options=['detect_retina', 'min_zoom', 'tile_size', 'attribution', 'opacity', 'max_zoom'], url='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'), Polyline(color='#2C3D49', fill_opacity=0.0, locations=[[49.54950147, 8.449381135], [49.4889, 8.544410000000001], [49.4697, 8.56625], [49.47516629999999, 8.60604], [49.4749374, 8.658517999999999], [49.45155, 8.667119999999999], [49.4296, 8.68253], [49.39979, 8.69048], [49.354684000000006, 8.679659], [49.3293, 8.693337], [49.303295, 8.73808], [49.223374, 8.653619], [49.1553, 8.625982], [49.13613, 8.604289999999999], [49.0457, 8.526244], [48.982766, 8.549681], [48.953148999999996, 8.570406], [48.95221, 8.65925], [48.9097366, 8.718982], [49.0498, 8.734470000000002], [49.08014, 8.549252000000001], [49.14754, 8.58806], [49.18042, 8.5478], [49.211945, 8.527593], [49.251178741455, 8.5408201217651], [49.291956, 8.566597], [49.315259999999995, 8.577630000000001], [49.3603, 8.526795], [49.3798, 8.58478], [49.480352, 8.479399], [49.47509, 8.4874], [49.54950147, 8.449381135], [49.4889, 8.544410000000001], [49.4697, 8.56625], [49.47516629999999, 8.60604], [49.4749374, 8.658517999999999], [49.45155, 8.667119999999999], [49.4296, 8.68253], [49.39979, 8.69048], [49.354684000000006, 8.679659], [49.3293, 8.693337], [49.303295, 8.73808], [49.223374, 8.653619], [49.1553, 8.625982], [49.13613, 8.604289999999999], [49.0457, 8.526244], [48.982766, 8.549681], [48.953148999999996, 8.570406], [48.95221, 8.65925], [48.9097366, 8.718982], [49.0498, 8.734470000000002], [49.08014, 8.549252000000001], [49.14754, 8.58806], [49.18042, 8.5478], [49.211945, 8.527593], [49.251178741455, 8.5408201217651], [49.291956, 8.566597], [49.315259999999995, 8.577630000000001], [49.3603, 8.526795], [49.3798, 8.58478], [49.480352, 8.479399], [49.47509, 8.4874]], options=['clickable', 'fill_opacity', 'no_clip', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'smooth_factor']), Circle(color='#00B7FF', fill_color='#00B7FF', fill_opacity=0.5, location=[49.54950147, 8.449381135], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3), Circle(color='#00B7FF', fill_color='#00B7FF', fill_opacity=0.5, location=[49.4889, 8.544410000000001], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3), Circle(color='#00B7FF', fill_color='#00B7FF', fill_opacity=0.5, location=[49.4697, 8.56625], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3), Circle(color='#8DC7B1', fill_color='#8DC7B1', fill_opacity=0.5, location=[49.47516629999999, 8.60604], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3), Circle(color='#8DC7B1', fill_color='#8DC7B1', fill_opacity=0.5, location=[49.4749374, 8.658517999999999], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3), Circle(color='#C1FF00', fill_color='#C1FF00', fill_opacity=0.5, location=[49.45155, 8.667119999999999], opacity=1.0, options=['fill_opacity', 'line_join', 'weight', 'class_name', 'stroke', 'line_cap', 'fill', 'dash_array', 'color', 'opacity', 'fill_color', 'pointer_events', 'clickable'], radius=500, weight=3)

### Legend
```green``` -  This is the cheapest gas station within the capacity of your tank, fill up as much as possible

```blue``` -  We will run out of gas util we reach the next station, so fill up enough to go to the next cheapest station

```gray``` - Don't stop